In [1]:
%load_ext autoreload
%env CUDA_VISIBLE_DEVICES = 2 python benchmark.ipynb

env: CUDA_VISIBLE_DEVICES=2 python benchmark.ipynb


In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import gc

from ptflops import get_model_complexity_info

%autoreload 2
pd.set_option("display.precision", 2)

In [3]:
device = 'cuda'
device

'cuda'

In [4]:
class Time_tracker:
    def __enter__(self):
        self.start = torch.cuda.Event(enable_timing=True)
        self.end = torch.cuda.Event(enable_timing=True)
        self.start.record()
        return self

    def __exit__(self, type, value, traceback):
        self.end.record()
        torch.cuda.synchronize()
        
    def timer(self):
        
        return self.start.elapsed_time(self.end)

In [5]:
class Memory_tracker:
    def __enter__(self, device=device):
        self.device=device
        self.start_memory = get_memory(device=self.device)
        
        return self

    def __exit__(self, type, value, traceback):
        self.end_memory = get_memory(reset_memory=False, device=self.device) 
        
    def memory_allocated(self):
        
        return float(self.end_memory - self.start_memory)

In [6]:
def get_memory(reset_memory=True, device=None):
    """Take current max allocated memory, either with or without resetting"""
    if reset_memory:
        torch.cuda.reset_peak_memory_stats()
            
    max_memory = torch.cuda.max_memory_allocated(device)
        
    return max_memory
    
    
def make_initialization_inputs(inputs, device=None):
    """ Take either tensor, shape tuple or list of them, and always return tensor or list of them. """
    if isinstance(inputs, torch.Tensor):
        pass
    elif isinstance(inputs, tuple):
        inputs = torch.rand(*inputs, device=device)
    elif isinstance(inputs, list):
        inputs = [make_initialization_inputs(item, device=device) for item in inputs]
    return inputs

    
    
def tracker(module, inputs, repeats=300, warmup=40, device=None, track_backward=True, channels_last=False, amp=False) -> dict:
    """Track module #macs, #parameters, time and memory consumption on forward and backward pass for a given inputs tensor or inputs shape"""
    
    with Time_tracker() as total_time:
    
        result = {}

        torch.cuda.empty_cache()

        inputs = make_initialization_inputs(inputs=inputs, device=device)
        module.to(device)

        if channels_last:
            inputs.to(memory_format=torch.channels_last)
            module.to(memory_format=torch.channels_last)          

        forward_timings = []
        backward_timings = []


        for i in range(repeats + warmup):

            with torch.cuda.amp.autocast(enabled=amp):
                if i < warmup:
                    outputs = module(inputs)
                    outputs.backward(outputs)
                    del outputs
                    torch.cuda.empty_cache()
                    i += 1
                    continue
                # calculate forward operation time  
                with Time_tracker() as t:
                    outputs = module(inputs)

                forward_time = t.timer()        
                forward_timings.append(forward_time) 

            if track_backward:
                # calculate backward operation time 
                with Time_tracker() as t:
                    outputs.backward(outputs)
                backward_time = t.timer()
                backward_timings.append(backward_time)

        result['forward time mean(ms)'] = np.mean(forward_timings)
        result['forward time std(ms)'] = np.std(forward_timings)

        # calculate forward memory
        with Memory_tracker() as memory:
            module(inputs)
        forward_memory = memory.memory_allocated()
        result['forward memory(MB)'] = forward_memory / 2**20

        if track_backward:
            result['backward time mean(ms)'] = np.mean(backward_timings)
            result['backward time std(ms)'] = np.std(backward_timings)
            
            # calculate backward memory
            outputs = module(inputs)
            with Memory_tracker() as memory:
                outputs.backward(outputs)
            backward_memory = memory.memory_allocated()
            result['backward memory(MB)'] = backward_memory / 2**20

        macs, params = get_model_complexity_info(module, tuple(inputs.shape[1:]), as_strings=False, print_per_layer_stat=False)
        result['macs'] = macs
        result['parameters'] = float(params)

    result['time total(ms)'] = total_time.timer()
    
    return result

In [7]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [8]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory(MB)','backward memory(MB)',
                                                        'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True)

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory(MB),backward memory(MB),macs,parameters,time total(ms)
conv_64_512_1x1,0.44,0.04,0.58,0.04,32.22,0.12,545259520.0,33280.0,1635.65
conv_64_512_3x3,0.83,0.04,1.29,0.02,35.13,1.12,4690151424.0,295424.0,800.75
bottleneck_64_512_3x3,0.89,0.05,1.54,0.03,35.57,2.56,2383218688.0,150048.0,884.58
conv_64_512_3x3_g2,1.91,0.04,2.63,0.03,33.23,3.06,11956733952.0,803328.0,1602.94
conv_64_512_3x3_g8,0.67,0.03,1.46,0.04,31.46,0.77,2994898944.0,201216.0,826.34
conv_padding,0.76,0.05,0.96,0.05,35.13,87.12,4840226816.0,295424.0,717.55
conv_nn_Padding,0.84,0.03,1.29,0.03,39.38,1.12,4840226816.0,295424.0,782.09


In [9]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory(MB)','backward memory(MB)',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True)

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory(MB),backward memory(MB),macs,parameters,time total(ms)
conv_64_512_1x1,0.48,0.03,0.4,0.1,36.47,34.0,545259520.0,33280.0,408.12
conv_64_512_3x3,1.01,0.03,1.36,0.05,37.22,34.0,4690151424.0,295424.0,901.33
bottleneck_64_512_3x3,0.93,0.05,2.37,0.06,39.65,34.0,2383218688.0,150048.0,1281.16
conv_64_512_3x3_g2,1.9,0.03,13.6,0.48,37.42,39.2,11956733952.0,803328.0,5405.86
conv_64_512_3x3_g8,1.08,0.03,12.23,0.08,35.64,34.77,2994898944.0,201216.0,4641.64
conv_padding,1.0,0.03,2.09,0.05,38.19,34.0,4840226816.0,295424.0,1147.16
conv_nn_Padding,1.08,0.05,1.42,0.47,42.25,34.0,4840226816.0,295424.0,992.13


In [10]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory(MB)','backward memory(MB)',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, amp=True)

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory(MB),backward memory(MB),macs,parameters,time total(ms)
conv_64_512_1x1,0.5,0.03,0.69,0.06,36.22,34.0,545259520.0,33280.0,539.07
conv_64_512_3x3,0.53,0.03,0.7,0.06,37.01,34.0,4690151424.0,295424.0,538.1
bottleneck_64_512_3x3,0.82,0.04,1.3,0.08,37.66,34.0,2383218688.0,150048.0,905.02
conv_64_512_3x3_g2,10.7,0.05,12.09,0.04,36.31,39.2,11956733952.0,803328.0,7885.11
conv_64_512_3x3_g8,2.53,0.03,10.09,0.09,34.53,34.77,2994898944.0,201216.0,4392.98
conv_padding,0.61,0.05,0.76,0.3,38.0,34.0,4840226816.0,295424.0,595.11
conv_nn_Padding,0.71,0.3,0.75,0.08,42.25,34.0,4840226816.0,295424.0,665.79


In [11]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory(MB)','backward memory(MB)',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, amp=True)

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory(MB),backward memory(MB),macs,parameters,time total(ms)
conv_64_512_1x1,0.48,0.03,0.6,0.12,36.22,34.0,545259520.0,33280.0,496.09
conv_64_512_3x3,0.58,0.03,0.65,0.11,37.01,34.0,4690151424.0,295424.0,542.08
bottleneck_64_512_3x3,0.82,0.04,1.31,0.08,37.66,34.0,2383218688.0,150048.0,916.4
conv_64_512_3x3_g2,10.66,0.05,12.1,0.04,36.31,39.2,11956733952.0,803328.0,7865.41
conv_64_512_3x3_g8,2.56,0.03,10.12,0.06,34.53,34.77,2994898944.0,201216.0,4405.01
conv_padding,0.61,0.05,0.75,0.25,38.0,34.0,4840226816.0,295424.0,591.44
conv_nn_Padding,0.69,0.06,0.75,0.07,42.25,34.0,4840226816.0,295424.0,662.27
